In [ ]:
# Author: Zhengxiang (Jack) Wang 
# Date: 2021-07-28
# GitHub: https://github.com/jaaack-wang 
# About: Training word embeddings using skip-gram with negative sampling in paddle

# Overview

In this notebook, we will train word embeddings employing Skip-gram with negative sampling algorithms as described by [Mikolov et. al. (2013a)](https://arxiv.org/pdf/1301.3781.pdf) and [Mikolov et. al. (2013b)](https://papers.nips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf). More concretely, we will learn the steps (e.g., text normalization, subsampling, negative sampling, and mini-batches) needed to preprocess a corpus and how to use `paddle` to construct a skip-gram neural network model to train word embeddings out of the preprocessed corpus.

<br>

Related contents: <br>

- [loading pre-trained word embedding in paddlenp](https://colab.research.google.com/drive/1WSyYtDiHwXe4MFTwe_X6hQ5atBqNsFax?usp=sharing)
- [calculating text cosine similarity in paddlenlp](https://colab.research.google.com/drive/1QYSJ3x6Ap5HG8O4R4yqAyw6iq18JahdO?usp=sharing)
- [embedding visualization using paddlepaddle tool](https://colab.research.google.com/drive/1B9pcYR9fVvmB1pPWiIqb0u_WmxlY--T8?usp=sharing)
- [embedding visualization using tensorflow tool](https://colab.research.google.com/drive/1HZdDA_TzdJhGo_uIUSa84rP6yQjHvMT3?usp=sharing)


<br>


<table align="right">
  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/16xghJX6WGEdlY3f2MamI5W7SJ6cniW4U?usp=sharing"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab </a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/jaaack-wang"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> Author's GitHub </a>
  </td>
  <td>
    <a href="https://docs.google.com/uc?export=download&id=16xghJX6WGEdlY3f2MamI5W7SJ6cniW4U"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download this notebook </a>
  </td>
</table> 


<br>


# Table of Contents
- [1. Word representation and word embedding](#1)
  - [1.1 Word representation](#1-1)
  - [1.2 General model to learn word embeddings](#1-2)
  - [1.3 Skip-gram and negative sampling](#1-3)
- [2. Word embedding training using skip-gram with negative sampling algorithms](#2)
  - [2.1 General procedure](#2-1)
  - [2.2 Data loading and preprocessing](#2-2)
  - [2.3 Model configuration](#2-3)
  - [2.4 Training](#2-4)
- [3. Things to do next](#3)
  - [3.1 Save the trained embeddings](#3-1)
  - [3.2 Visualization](#3-2)
  - [3.3 Word analogy test](#3-3)
- [4. References](#4)

In [ ]:
# first, make sure that you have paddlepaddle installed
!pip install --upgrade paddlepaddle

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |███████████████████▋            | 66.7MB 8.9MB/s eta 0:00:05Requirement already satisfied, skipping upgrade: astor in /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages (from paddlepaddle) (0.8.1)


**Dependencies**

In [ ]:
import os
import requests
from collections import Counter 
import math
import random
import numpy as np
import paddle
from paddle.nn import Embedding
import paddle.nn.functional as F

<a name='1'></a>
# 1. Word representation and word embedding

<a name='1-1'></a>
### 1.1 Word representation

One-hot encoding and word embedding are one of two commonest ways to represent words in machine(/deep)-learning-based natural language processing. In both approches, a word is represented by a vector that has certain dimensions $N$(number of elements in the vector) and a vocabulary list can thus be represented as a vocabulary size $V$ by dimensions $N$ matrix (i.e., $R^{V*N}$).

<br>

For one-hot encoding, a word vector is always made up of one 1 and many 0s such that the sum of each vector all equals 1, as exemplified by the figure below. This approach is very easy to implement, but does not consider any potential relations among words as the relation between any two one-hot encoded words is nothing but identical (e.g., the distance between two word vectors is always $\sqrt{2}$). Moreover, it can also be computationally expensive to perform matrix multiplication as one-hot encoding requires the word matrix to be $V$ by $V$ (i.e., $N = V$) dimensions, provided that a real-world NLP problem usaully has tens of thousands or even larger vocabulary size. 

<img src='https://i.gzn.jp/img/2018/10/12/language-translator-deep-learning/008.png' height='400' width='600'>

In contrast, word embedding is a much more economic and efficient way to represent words as it uses much smaller $N$ to be the dimension of each vector, which typically ranges from 50 to 300. Instead of using only 1 and 0, word embedding uses floating numbers to fill in each word vector such that each word vector has different relations with other word vectors. Ideally, similar words should have closer relations and dissimilar words should have more distant relations (usaully in terms of their **cosine similarity** score that measures the angel between two vectors). It has also been shown by [previous research](https://arxiv.org/pdf/1301.3781.pdf) that two word pairs that has similar internal relation (e.g., semantic, syntactic) should have large cosine similarity score (or visually, parallel to each other), such as the classic example: $women \rightarrow queen \approx man \rightarrow king$. The figure below is a visual illustration of what ideal word embeddings should look like in a 2-D dimension-reduced plot. 
 

![](https://ai-studio-static-online.cdn.bcebos.com/00ba55f7304e4f97942165cf1deb946ced404a19325d40969b7c220e30cf527e)

<br>

Similar to tokenization to text processing, word embedding has also been recgonized as one of the most fundamental technique in natuural language processing and is essential for many downstream machine(/deep)-learning-based natural language processing tasks, such as Named Entity Recognition, Information Retrieval, and Machine Translation etc.

<a name='1-2'></a>
### 1.2 General models to learn word embeddings

To train word embeddings, three models are widely used, namely [Continuous Bag-of-Words (CBOW) model, Skip-gram model](), and [Global Vectors for Word Representation (GloVe) model](https://nlp.stanford.edu/pubs/glove.pdf). According to Google Scholar, the papers where these models were origianlly proposed all have over 20,000 citations. The seminal work by [Bengio et. al., 2003](https://www.jmlr.org/papers/volume3/tmp/bengio03a.pdf), which proposes a Nerual-Network-based probabilistic Language Model (NNLM), can be seen as a prototype for CBOW and Skip-gram models as they all rely on local context and shallow neural network to train word embeddings. GloVe is different than these models in that it additionally leverages global word-word co-occurrence counts to make up for the constrains of the previous models only looking at local contexts. 

<a name='1-3'></a>
### 1.3 Skip-gram model architecture and negative sampling

As mentioned above, Skip-gram model is trained on the local contexts of target words, just like CBOW model. But like the latter, which uses the nearby words (contexts) of target words to predict the target words, the Skip-gram model uses the target words to predict their nearby words. Nearby words are usually defined by a window size that spans around the target words. For example, if the window size is 3, then the contexts are words that are in both the left side and the right side of the target words up to 3 words. The model architectures of these two models can be illustrated by the following figure, which shows that both models are made up of an input layer, a hidden layer and an output layer, a simple shallow neural network structure. 

<img src='https://drive.google.com/uc?export=view&id=1jw9Tae0avHcWY7RKrxigOMuq8f-kbWsk'>

<br>

For Skip-gram model, the training process goes like this: the input layer is a 1 by $V$ (vocabulary size) **one-hot** row vector, which are then multiplied by a $V$ by $N$ (i.e., number of vector dimensions or features) weight matrix that would result in a 1 by $N$ row vector in the hidden layer. This row vector is then multiplied by another $N$ by $V$ weight matrix followed by a softmax function that will return a 1 by $V$ row vector. This row vector stores the probabilities of the context wors for the target word. To optimize our model, we want to make sure that the top the top $C$ candidates $C$ (i.e., number of context words to predict) match with the real context words as much as possible by minimizing the loss of our model during backpropagation. The architecture of the Skip-gram model is shown in the figure below. 

<img src='http://www.claudiobellei.com/2018/01/06/backprop-word2vec/Skipgram.png' width='400' height='400'>

<br>

Please note that the operation between the input layer and the hidden layer is just a **simple linear activation** that takes the form of $A^{(1)} = Z^{(1)} = xW^{(1)}$. <ins>**This $V$ by $N$ weight matrix is actually the word embeddings we aim to optimize and obtain for the entire vocab!**</ins> The linear activtaion can be seen as a transformation that extracts a target word embedding from the embedding matrix because the multiplication between the input one-hot vector and the embedding weight matrix will locate and return a row vector of the embedding matrix whose position matches with the postion of 1 in the one-hot vector. 

<br>

Moreover, since the vocabulary size for a corpus which we can train word embeddings on is usually very large, the last step of the calculation between the hidden layer and the output is very computational costly. To solve this problem, multiple methods have been proposed, such as [hierarchical softmax](https://ruder.io/word-embeddings-softmax/) and negative sampling. For example, the negatve sampling method will randomly select $K$ non-context words from the vocabulary for every positive context word of the target word to form comparisons. As a result, we can change the Skip-gram model from predicting context words from a large vocabulary into one that only determines whether the context word is a real context word or a just negative sample for a limited amount of times (number of context words plus $K$ times of the context words). Therefore, the last weight matrix can be reduced to a $N$ and 1 vector and the last activation function in the output can be a sigmoid function. This will significantly speed up the entire training process. 

<a name='2'></a>
# 2. Word embedding training using skip-gram and negative sampling algorithms

<a name='2-1'></a>
### 2.1 General procedure

There are three steps we need to take to train word embeddings: 
- First, we need to load a corpus that we train word embedding on and preprocess the corpus so that it can be trained. 
- Then, we need to build the Skip-gram model to train the preprocessed corpus. In this notebook, we will use `paddle` to construct the model. 
- Lastl, we need to train word embeddings using the corpus and the model by minimizing the prediction loss for the context words. 

<br>

In the following parts of the notebook, we will mostly spend time preprocessing the corpus because utilizing `paddle` package frees us from writing algorithms for the Skip-gram model  from bottom up. Training the model only takes time, so we will not discuss it too much either. 

<br>

Moreover, it should also be noted that, with various free deep learning frameworks available that can save us from building deep learning models with lengthy code, preprocessing a corpus is often actually more important than building those models. 

<a name='2-2'></a>
### 2.2 Data loading and preprocessing

<a name='2-2-1'></a>
#### 2.2.1 Data loading
- We will use a corpus of 100 MB collected from wikipedia to train word embeddings. 
- This corpus is in a way classic as it was also used by Mikolov as demo text in his original [word2vec project](https://github.com/tmikolov/word2vec)
- We will build a simple function to download the file from a given url. 

In [ ]:
def get_file_from_url(url, file_name, file_dir='./'):
  '''A simple function that downloads file from url and returns 
  the saved file path.
  '''
  file_path = file_dir + file_name

  if os.path.exists(file_path):
    return file_path
  
  r = requests.get(url)
  try:
    content = r.content
  except:
    raise FileNotFoundError('File not found or cannot be downloaded.')

  with open(file_path, 'wb') as f:
    f.write(content)
  f.close()
  
  print(file_name + ' saved in ' + file_path)
  return file_path

In [ ]:
file_path = get_file_from_url(
    url='https://dataset.bj.bcebos.com/word2vec/text8.txt',
    file_name = 'text8.txt')

# read the corpus
text8 = open(file_path, 'r').read()
# print the first 100 characters 
# the corpus has already been unpunctuated,
# so we can tokenize it by simply splitting it by spaces.
print('\nThe first 300 characters of text8: \n\n', text8[:300])


The first 300 characters of text8: 

  anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the organiz


<a name='2-2-2'></a>
#### 2.2.2 Data preprocessing
**First steps:**
- We will need to first tokenize the corpus and get the vocabulary out of it.
- Then, we will create the following three dictionaries:
  - `word2id_dict`, which maps words to their ids, will be used to convert the corpus into integers for ease of computer processing;
  - `id2word_dict`, which maps word ids to words, will be used as lookup table to convert word ids back to words;
  - `word2id_fdist`, which stores the frequency distribution of words (represented by theird ids), will be used for subsumpling.
- In theory, the word ids can be assigned according to the occurrence of new words in the corpus ('first come first serve'), but in practice they are usually in the descending order of word frequency. In other words, the more frequently a word occurs, the lower its id will be. This arrangement makes the word ids more meaningful and easier to manage. 


In [ ]:
# tokenize the corpus 
# the corpus is already preprocessed, but it 
# never hurts to strip extra spaces and lower case
# the corpus before tokenization
tokens = text8.strip().lower().split()
print('The first 50 words of the corpus:')
print(tokens[:50])

The first 50 words of the corpus:
['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the']


In [ ]:
# create the dictionaries

# we will first create a word_fdist that maps 
# words to their frequency from highest to lowest
word_fdist = dict(Counter(tokens).most_common())

# create the three dictionaries
word_to_id = {}
id_to_word = {}
word2id_fdist = {}

# we use integer indices (ids) to represent words
id = 0
for word, freq in word_fdist.items():
  word_to_id[word] = id
  id_to_word[id] = word
  word2id_fdist[id] = freq
  id += 1

print('vocab size of the corpus: ', len(word_to_id))

vocab size of the corpus:  253854


In [ ]:
# let's randomly check 10 words along with their ids and frequency

random_ids = [23, 768, 456, 5677, 6879, 987, 234435, 234, 4522, 78633]
tmp = '{0}\t\t{1:10}\t{2}'
print(tmp.format('id', 'word', 'frequency'))
for id in sorted(random_ids):
  print(tmp.format(id, id_to_word[id], word2id_fdist[id]))

id		word      	frequency
23		with      	95603
234		popular   	5967
456		change    	3483
768		effects   	2194
987		conditions	1805
4522		emotional 	362
5677		possess   	273
6879		paths     	211
78633		docu      	4
234435		recrystallizes	1


**Further steps:**
- Now, with the dictionaries ready, we want to convert the entire corpus into integers using the `word_to_id`. 
- As we can see above, the frequency between frequently used words and infrequently used is very imbalanced. The [Zipf's law](https://en.wikipedia.org/wiki/Zipf%27s_law) states that the rank-frequency distribution is an inverse relation on a log-log scale for natural languages, something like the following. 

<img src='https://upload.wikimedia.org/wikipedia/commons/a/ac/Zipf_30wiki_en_labels.png' alt='A plot of the rank versus frequency for the first 10 million words in 30 Wikipedias in a log-log scale.' width='600' height='400'>

- Compared to infrequent words, highly frequent words usually contain less contextual information. Scholars (e.g., [Mikolov et. al., 2013b](https://papers.nips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf)) have proposed to reduce the occurence of highly used words by means of subsampling so that a target word can learn much more meaningful word embedding during training. The subsampling formula is given by a probability function that filters frequent words:

$$P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}}$$

where $f(w_i)$ is the frequency of word $w_i$ (divided by the size of the corpus) and t is a chosen threshold, typically around $10^{-5}$. For example, for $t=10^{-5}$, if a word occurs 1 per 1,000 words ($f=10^{-3}$), then the possibility of it being filtered equals 

$$1 - \sqrt{\frac{10^{-5}}{10^{-3}}} = 1 - \sqrt{0.01} = 0.9$$

whereas a word with frequency of 1 per $10^5$ words ($f=10^{-5}$) will never be discarded as $P(w_i)$ will equal 0 in this case. Therefore, the greater $f(w_i)$ is than the threshold $t$, the higher probability it will get reduced.  

<br>

- Subsampling will aggressively subsample words whose frequency is greater than $t$. Let's say the most frequently used word occurs 5 per 100 words, then it will be reduced to 1.4% of its original size with $t=10^{-5}$. In a big corpus (e.g., billion words), even with such great reduction, this 1.4% will still be much larger than those low frequency words whose frequencies do not pass $t=10^{-5}$, so that the original ranking of the frequencies can be preserved. However, for a small corpus, such as one of 100,000 words, such reduction will change the original ranking and severely damage the original linguistic information of the corpus. Therefore, a good threshold is not an unimportant decision to make.

- As subsampling reduces the corpus size, it also speeds up the computation for training word embeddings. 

- Subsampling might not be entirely necessary as the imbalanced nature of word usage in natural languages can also be something a word embedding algorith should learn. 

In [ ]:
# convert the corpus into integer indices using word2id_dict
text8 = [word_to_id[word] for word in tokens]
print('First 50 "words" of the converted corpus:')
print(text8[:50])
# also check the corpus size before subsampling
print('\nCorpus size before subsampling: ', len(text8))

First 50 "words" of the converted corpus:
[5233, 3080, 11, 5, 194, 1, 3133, 45, 58, 155, 127, 741, 476, 10571, 133, 0, 27349, 1, 0, 102, 854, 2, 0, 15067, 58112, 1, 0, 150, 854, 3580, 0, 194, 10, 190, 58, 4, 5, 10712, 214, 6, 1324, 104, 454, 19, 58, 2731, 362, 6, 3672, 0]

Corpus size before subsampling:  17005207


In [ ]:
# define the subsampling function
# we use random and math because they run faster than the related numpy functions

def subsample(converted_text, word2id_fdist, threshold=1e-5, seed=323):
  '''Subsamples the converted corpus. 

  Args:
    converted_text (list): text converted into a list of integer indices.
    word2id_fdist(dict): frequency distribution disctionary of words represented by ids.
    threshold (float): a threshold float for the discard possibility function.

  Returns: subsampled text (a list of integer indices.)
  '''
  def to_discard(word):
    # a / (b / c) = a / b * c
    p = 1 - math.sqrt(threshold / word2id_fdist[word] * size)
    return random.random() < p
  
  # the seed is just for the replicability of this notebook 
  random.seed(seed)
  size = len(converted_text)
  # word here means word_id
  return [word for word in converted_text if not to_discard(word)]

In [ ]:
# our corpus only has 17 million words, 
# so we will choose a larger threshold value (1e-4 instead of 1e-5)
# meaning we will only subsample words that occurs over 1 time per 10000 words
# this reduces our corpus by 1/2 of the original size!
text8_sub = subsample(text8, word2id_fdist, 1e-4)
print('Subsampled corpus size:, ', len(text8_sub))
print('\nFirst 50 words of the subsampled corpus:')
print([id_to_word[id] for id in text8_sub[:50]])

Subsampled corpus size:,  8743856

First 50 words of the subsampled corpus:
['anarchism', 'originated', 'term', 'abuse', 'first', 'against', 'working', 'radicals', 'including', 'diggers', 'revolution', 'sans', 'culottes', 'of', 'the', 'revolution', 'whilst', 'pejorative', 'way', 'describe', 'any', 'act', 'violent', 'means', 'destroy', 'organization', 'has', 'taken', 'positive', 'label', 'by', 'self', 'anarchists', 'anarchism', 'derived', 'greek', 'archons', 'ruler', 'chief', 'king', 'anarchism', 'as', 'political', 'philosophy', 'belief', 'rulers', 'unnecessary', 'and', 'should', 'abolished']


**Final steps:** <br>

- With the original corpus converted into integer indices and subsampled, we can now build a dataset for training our word embedding model. As we use negative sampling to obtain non-context words (labelled as 0) for a particular word in a sequence (other words in the sequence are the context words to be predicted), for every training example, we need a `target word`, a `context word` and a `label` to indicate whether the `context` appears in the same sequence of text with the `target word`.
- To speed up our training process, we will use mini-batch gradient descent instead of batch gradient descent to learn word embeddings. We will build a function to split the corpus into mini bacthes. 

In [ ]:
# define a function to generate the training dataset that contains
# a list of (target word, context word, label) tuples

def get_training_dataset(corpus, window_size, num_ns, vocab=None, seed=654): 
  '''Generates a dataset for word embedding training using skip gram 
  with negative sampling. 

  Args:
    - corpus (list): a list of integer indices that represent words.
    - window_size (int): the max distance for a context word
    - num_ns (int): number of negative examples to sample
    - vocab (list, set): a list or set of integer incides that denote the vocab of 
        given corpus. If not given, vocab = set(corpus).
    - seed(int): defaults to 654, used only for the replicability of this notebook 

  Returns:
    A dataset that contains a list of ((target word, context word), label) tuples.
  '''

  random.seed(seed)

  if vocab is None:
    vocab = set(corpus)

  dataset = []
  # we will scan every word in the corpus to get context words and negative samples
  for idx, word in enumerate(corpus):
    left_context = corpus[idx-window_size if idx-window_size>= 0 else 0: idx]
    right_context = corpus[idx+1: idx + window_size + 1]
    contexts = left_context + right_context
    for context in contexts:
      dataset.append((word, context, 1))

      # for every positive example, we need num_ns negative examples
      
      # although we do not want to sample two or more identical negative examples, 
      # as the vocab size is big enough, the ns_chosen part of code is only optional 
      ns_chonsen = []
      for _ in range(num_ns):
        neg_example = random.choice(vocab)
        while neg_example in contexts or neg_example in ns_chonsen:
          neg_example = random.choice(vocab)
        else:
          dataset.append((word, neg_example, 0))
          ns_chonsen.append(neg_example)

  return dataset


<a name='reminder'></a>
<font color='red'>
<b>Reminder:</b>
<font color='black'>
If you are not runing this notebook on GPU, it is recommended that you use a much smaller training dataset by only selecting 10,000 words from the text8_sub below. 

In [ ]:
# If you are not running on a GPU that supports paddle, 
# you can only select 10,000 words from the text8_sub for practice
# Just so you know: it took me about 50 minutes to train the current model
# on GPU (Tesla V100) provided by Baidu's AI studio. 
# please also note that paddle gpu version cannot run on Google Colab
dataset = get_training_dataset(text8_sub[:4000000], 
                               window_size=3, 
                               num_ns=4, 
                               vocab=list(id_to_word.keys()))
tmp = '{0:20}{1:20}{2:15}'

# let's check the first 50 examples of our dataset
print(tmp.format('target word', 'context word', 'true context (1: yes; 0: no)'))
for i in range(50):
  print(tmp.format(id_to_word[dataset[i][0]],
                   id_to_word[dataset[i][1]], 
                   dataset[i][2]))

target word         context word        true context (1: yes; 0: no)
anarchism           originated                        1
anarchism           comsume                           0
anarchism           auerochse                         0
anarchism           behalten                          0
anarchism           channelised                       0
anarchism           term                              1
anarchism           militarev                         0
anarchism           grotzer                           0
anarchism           illite                            0
anarchism           ckis                              0
anarchism           abuse                             1
anarchism           nykyrka                           0
anarchism           walnut                            0
anarchism           tyseley                           0
anarchism           urbanski                          0
originated          anarchism                         1
originated          bohra          

**Make batches**

In [ ]:
def make_batches(dataset, batch_size=64, keep_reminder=True, seed=None):
  '''Convert a dataset into mini batches. 

  Args:
    dataset (list): should contains a list of tuples (target, context, label)
    batch_size (int): the maximum size for a batch. 
    keep_reminder (bool): Defaults to True. When set false, the last batch, if 
      smaller than the given batch_size, will not be included. 
    seed (int): defaults to None, used only for the replicability of this notebook  
  '''
  def read_dataset(start_idx, end_idx):
    targets, contexts, labels = [], [], []
    for t, c, l in dataset[start_idx: end_idx]:
      targets.append(t)
      contexts.append(c)
      labels.append(l)
    
    targets = paddle.to_tensor(targets, dtype="int64")
    contexts = paddle.to_tensor(contexts, dtype="int64")
    labels = paddle.to_tensor(labels, dtype="float32")
    return targets, contexts, labels
  
  if seed is not None:
    random.seed(seed)

  random.shuffle(dataset)
  data_size = len(dataset) 
  
  num_complete_batches = data_size // batch_size
  for n in range(num_complete_batches):
    yield read_dataset(n * batch_size, (n + 1) * batch_size)
  
  if keep_reminder and data_size % batch_size:
    yield read_dataset(num_complete_batches * batch_size, data_size)

In [ ]:
# check the batches
for batch in make_batches(dataset[:100], batch_size=30, seed=76):
  print(batch)

(Tensor(shape=[30], dtype=int64, place=CUDAPlace(0), stop_gradient=True,
       [3133, 3133, 45  , 5233, 5233, 3080, 3133, 3080, 3080, 194 , 3080, 3133, 194 , 3133, 194 , 3133, 194 , 194 , 45  , 3080, 3133, 3080, 3080, 5233, 5233, 194 , 194 , 45  , 3133, 3133]), Tensor(shape=[30], dtype=int64, place=CUDAPlace(0), stop_gradient=True,
       [55198 , 23149 , 114538, 194   , 116274, 177719, 23147 , 178101, 247675, 3080  , 5233  , 155   , 119722, 71781 , 112970, 5233  , 45    , 70000 , 201734, 194   , 136228, 41520 , 128407, 148229, 164179, 5233  , 72512 , 5658  , 98287 , 37569 ]), Tensor(shape=[30], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]))
(Tensor(shape=[30], dtype=int64, place=CUDAPlace(0), stop_gradient=True,
       [3080, 3133, 194 , 194 , 5233, 194 , 3080, 3133, 3133, 3080, 3133, 194 , 45  , 3133, 3133, 3080, 3080, 3080, 3080, 3133, 194 , 3133

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


<a name='2-3'></a>
### 2.3 Model configuration

The Skip-gram model (with negative sampling) configuration has already been explained in [1.3 Skip-gram model architecture](#1-3) and here is the quick recap:

<br>

- Input layer, which is a 1 by $V$ one-hot vector ($x$); 
- Hidden layer, a 1 by $N$ vector (i.e., $A^{(1)}$, the embedding for the input word, $N$ is the vector dimension) gotten through linear activitation: $A^{(1)} = Z^{(1)}  = xW^{(1)}$ where $W$ is a $V$ by $N$ matrix. 
- Output layer, a scalar (i.e., $y$) ranging from 0 to 1, gotten through sigmoid activitation: $y = \sigma(A^{(1)}W^{(2)})$ where $N$ is a $N$ by 1 vector. 

<br>

The reason why the last layer of the model uses sigmoid function instead of softmax function is due to the use of negative sampling, which turns the task of the model from predicting the most likely context words into deciding whether a given word is the context word or not. 

In [ ]:
#construct the Skip-gram model

#we need to inherit the paddle.nn.Layer for our model
class SkipGram(paddle.nn.Layer):

    def __init__(self, vocab_size, embedding_size, init_scale=0.1):
      '''
      Args: 
        vocab_size (int): the size of the vocabulary. 
        embedding_size (int): the size of the word embedding dimension. 
        init_scale (float): this defines the initial range of the word 
          embedding weights, i.e., [-init_scale, init_scale].  
      '''
      super(SkipGram, self).__init__()
      self.vocab_size = vocab_size
      self.embedding_size = embedding_size

      # we use Embedding inherited from paddle.nn.Layer to construct 
      # the word embeddings between the input layer and the hidden layer. 
      self.embedding = Embedding(
          num_embeddings = self.vocab_size,
          embedding_dim = self.embedding_size,
          weight_attr=paddle.ParamAttr(
              initializer=paddle.nn.initializer.Uniform(
                  low=-init_scale, high=init_scale)))
        
      # we use Embedding inherited from paddle.nn.Layer to construct 
      # the word embeddings between the hidden layer and the output layer. 
      self.embedding_out = Embedding(
          num_embeddings = self.vocab_size,
          embedding_dim = self.embedding_size,
          weight_attr=paddle.ParamAttr(
              initializer=paddle.nn.initializer.Uniform(
                  low=-init_scale, high=init_scale)))

    def forward(self, target, context, label):
      '''A callable function to do forward propagation. Simply call it by 
      SkipGram()(target, context, label).

      Args:
        target(tensor): integer, the target word used to predict context words.
        context(tensor): integer, the context word to be predicted. 
        label(int): 0 --> false ontext word. 1 --> true context word. 

      Returns:
        pred(float): the sigmoid output used to predict the label. 
        loss(float): the loss score. 
      '''
      # convert input into a word embedding: input layer --> hidden layer
      target_embd = self.embedding(target)
      context_embd = self.embedding_out(context)

      # hidden layer --> output layer followed by a sigmoid function
      word_sim = paddle.multiply(target_embd, context_embd)
      word_sim = paddle.sum(word_sim, axis=-1)
      word_sim = paddle.reshape(word_sim, shape=[-1])
      pred = F.sigmoid(word_sim)

      # we use binary_cross_entropy_with_logits to define the loss function
      loss = F.binary_cross_entropy_with_logits(word_sim, label) # word_sim but not pred here!
      loss = paddle.mean(loss)

      return pred, loss

<a name='2-4'></a>
### 2.4 Training


In [ ]:
# we can first define a get_similar_words function we can use during training
# so that we can monitor how the embeddings improve during training
def get_similar_words(query_word, k, embd, print_words=True):
  '''Returns the k most similar tokens to the query_token based on cosine similarity. 

  Args:
    query_word('str'): the query_word should be in the training dataset. 
    k (int): the number of most similar tokens to the query token. 
    embd (array-like): the embedding matrix.
    print_tokens(bool): defaults to True. If False, the most similar tokens will 
      not be printed out. 

  Returns:
    k most most similar tokens to the query_token. 
  '''
  W = embd.numpy() # embedding matrix as a numpy array
  x = W[word_to_id[query_word]] # the query token's embedding
  # 1e-9 is used to avoid zero division 
  cos = np.dot(W, x) / np.sqrt(np.sum(W * W, axis=1) * np.sum(x * x) + 1e-9)
  flat = cos.flatten()
  # we will not include the query token itself 
  k = k + 1
  indices = np.argpartition(flat, -k)[-k:]
  indices = indices[np.argsort(-flat[indices])]
  sim_words = [id_to_word[idx] for idx in indices][1:]
  if print_words:
    print('for word %s, the similar word is %s' % (query_word, sim_words))

  return sim_words

<font color='red'>
<b>Reminder:</b>
<font color='black'>

If you are not runing this notebook on GPU, it is recommended that you use a much smaller training dataset as noted in the previous [reminder](#reminder). You may also want to use a smaller batch size, a smaller embedding size (e.g., 20) to speed up the training process. Training word embeddings takes a lot of resources, so it is more important to know the process than training a good model yourself. If you are interested in training a larger model, you can run this notebook in [Baidu's AI studio](https://aistudio.baidu.com/aistudio/index).

<br>

For your convenience, you can download the trained embeddings by this notebook [in this zip file](https://drive.google.com/file/d/1KA14tazUiIEYQZ1BUjV15qg_FRY11lYW/view?usp=sharing). 

In [ ]:
# trainning 

# Hyperparameters
# if you run the following code, your results might be different 
# because of the randomization during th optimization process
batch_size = 256
epoch_num = 2
embedding_size = 50
step = 0
learning_rate = 0.001
vocab_size = len(id_to_word)

# Google Colab does not support this for some reason
# uncomment the following code if you run this on GPU
paddle.set_device('gpu:0')

# Skip gram model
skip_gram_model = SkipGram(vocab_size, embedding_size)
# We will use adam to optimize during the backpropagation 
adam = paddle.optimizer.Adam(learning_rate=learning_rate, parameters = skip_gram_model.parameters())

for epoch in range(epoch_num):
  for target, context, label in make_batches(dataset, batch_size):    
    # foward propagation 
      pred, loss = skip_gram_model(target, context, label)
    # paddle will automatically do the backpropagation for us
      loss.backward()
    # adam optimizer 
      adam.step()
    # clear the grads we already used.
      adam.clear_grad()

    # we will print the loss of our model every 10000 steps
      step += 1
      if step % 10000 == 0:
          print("epoch %d step %d, loss %.3f" % (epoch+1, step, loss.numpy()[0]))

    # we will manually check the performance of the model every 50000 steps by
    # seeing whether the similiar tokens to a query token make sense to us.  
      if step % 50000 ==0:
        embd_weights = skip_gram_model.embedding.weight
        print()
        get_similar_words('china', 5, embd_weights)
        get_similar_words('one', 5, embd_weights)
        get_similar_words('computer', 5, embd_weights)
        print()


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


epoch 1 step 10000, loss 0.426
epoch 1 step 20000, loss 0.189
epoch 1 step 30000, loss 0.221
epoch 1 step 40000, loss 0.285
epoch 1 step 50000, loss 0.255

for word china, the similar word is ['spirits', 'responses', 'fairly', 'makers', 'flowers']
for word one, the similar word is ['zero', 'nine', 'american', 'eight', 'two']
for word computer, the similar word is ['existed', 'creation', 'may', 'prayer', 'accepted']

epoch 1 step 60000, loss 0.271
epoch 1 step 70000, loss 0.162
epoch 1 step 80000, loss 0.247
epoch 1 step 90000, loss 0.296
epoch 1 step 100000, loss 0.342

for word china, the similar word is ['states', 'austria', 'baltic', 'denver', 'bulgarian']
for word one, the similar word is ['eight', 'nine', 'four', 'three', 'six']
for word computer, the similar word is ['application', 'color', 'does', 'user', 'models']

epoch 1 step 110000, loss 0.265
epoch 1 step 120000, loss 0.191
epoch 1 step 130000, loss 0.186
epoch 1 step 140000, loss 0.211
epoch 1 step 150000, loss 0.165

for 

<a name='3'></a>
# 3. Things to do next

<a name='3-1'></a>
### 3.1 Save the trained embeddings


In [ ]:
# let's save the words and their embeddings as tsv files

words = list(word_to_id.keys())
embeddings = skip_gram_model.embedding.weight

with open('text8_words.tsv', 'w') as f:
    f.write('\n'.join(words))
    f.close()

with open('text8_embds.tsv', 'w') as f:
    for embd in embeddings.numpy():
        f.write('\t'.join([str(em) for em in embd]))
        f.write('\n')
    f.close()

<a name='3-2'></a>
### 3.2 Visualization

You can refer to [embedding visualization using paddlepaddle tool](https://colab.research.google.com/drive/1B9pcYR9fVvmB1pPWiIqb0u_WmxlY--T8?usp=sharing)
and [embedding visualization using tensorflow tool](https://colab.research.google.com/drive/1HZdDA_TzdJhGo_uIUSa84rP6yQjHvMT3?usp=sharing) two files to learn how to visualize word embeddings. Below is the visualization of the word embeddings trained above projected by [TensorBoard's Embedding Projector](http://projector.tensorflow.org).

<img src='https://drive.google.com/uc?export=view&id=1N9z3_lTWgwhDZt-tzpI5fGs7SbiK-19D'>

<a name='3-3'></a>
### 3.3 Word analogy test

Word analogy is one of the most interesting application of word embeddings. Suppose we have three words, King, Queen, Women, and we want to find another word $W_4$ that makes the relations between Women and $W_4$ most similar to that between King and Queen. Word analysis is like finding the most similar word to another word, but at the word-pair level. 

<br>

As the trained word embeddings are not human readable, word analogy test has thus become a way to evaluate the performance of the trained embeddings. If you are interested, try to write a function to do just this. You may find this [questions-words.txt](https://github.com/tmikolov/word2vec/blob/master/questions-words.txt) corpus helpful for the evaluation. 

<a name='4'></a>
# 4. References
- [Mikolov et. al., 2013a. Efficient estimation of word representations in vector space](https://arxiv.org/pdf/1301.3781.pdf)
- [Mikolov et. al., 2013b. Distributed representation of words and phrases and their compositionality](https://papers.nips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf)
- [Source code in C by Mikolov](https://github.com/tmikolov/word2vec)
- [Pennington et.al., 2014. GloVe- Global Vectors for Word Representation](https://nlp.stanford.edu/pubs/glove.pdf)
- [On word embeddings - Part 2: Approximating the Softmax](https://ruder.io/word-embeddings-softmax/)
- [Sequence Models (Course 5) in Deep Learning Specialization taught by Andrew Ng on Coursera](https://www.coursera.org/specializations/deep-learning?).
- [Marginalia, 2018. The backpropagation algorithm for Word2Vec](http://www.claudiobellei.com/2018/01/06/backprop-word2vec/)
- [Rong, 2014. Word2vec Parameter Learning Explained](https://arxiv.org/pdf/1411.2738.pdf)